In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import time
import os
import copy
import csv
import pandas as pd
import random

from PIL import Image

# 대화 모드

plt.ion()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

## 데이터셋

In [ ]:
# 이미지 리사이즈 및 정규화
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((200,200)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.RandomGrayscale(p=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]),
])

# dataset 및 dataloader
#target_dir = "../input/track02-dataset/hallym-2020-2-aicomp02/train"
target_dir = "../input/realfinal/train"
dataset = torchvision.datasets.ImageFolder(root=target_dir,
                                           transform=transform)
class_names = dataset.classes

dataset_sizes = {}
dataset_sizes["train"] = int(0.8 * len(dataset))
dataset_sizes["val"] = len(dataset) - dataset_sizes["train"]

datasets = {}
datasets["train"], datasets["val"] = torch.utils.data.random_split(
    dataset, [dataset_sizes["train"], dataset_sizes["val"]])

dataloaders = {}
dataloaders["train"] = torch.utils.data.DataLoader(datasets["train"],
                                                   batch_size=16,
                                                   shuffle=True,
                                                   num_workers=8)
dataloaders["val"] = torch.utils.data.DataLoader(datasets["val"],
                                                   batch_size=8,
                                                   shuffle=True,
                                                   num_workers=2)

for x in ["train", "val"]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
print(class_names)

## Utils

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([.5, .5, .5])
    std = np.array([.5, .5, .5])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# 학습 데이터의 배치
inputs, classes = next(iter(dataloaders['val']))

# 배치로부터 격자 형태의 이미지
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval() # 평가 모드; batch norm, dropout, ... X
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'ground truth:{class_names[labels[j]]}, predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## 모델생성

In [ ]:
def eval_model(model, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    for i, data in enumerate(dataloaders["val"]):
        model.eval()
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)
        
    avg_loss = loss_test / dataset_sizes["val"]
    avg_acc = acc_test.double() / dataset_sizes["val"]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))

### EfficientNet Model

In [ ]:
pip install geffnet

In [ ]:
import geffnet

#ef_model = geffnet.efficientnet_b3(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
ef_model = geffnet.mixnet_xl(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
ef_model.train()

In [ ]:
import geffnet

ef_model = geffnet.mixnet_xl(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
ef_model.train()

# 이전 레이어 학습 정지
for param in ef_model.parameters():
    param.require_grad = True #원래 False

    
# 새로운 레이어 추가
num_features = ef_model.classifier.in_features
features = list(ef_model.classifier.children())[:-1]
features.extend([nn.Linear(num_features, len(class_names))])
ef_model.classifier = nn.Sequential(*features) # classifier 변경
print(ef_model)

In [ ]:
ef_model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(ef_model.parameters(), lr=0.0001, weight_decay=1e-5)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## 훈련전 모델 평가 및 시각화

In [ ]:
print("Test before training")
eval_model(ef_model, criterion)
visualize_model(ef_model)

## 훈련

In [ ]:
from tqdm import tqdm

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        # training
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        model.train(True)
        
        # 데이터 반복
        for i, data in tqdm(enumerate(dataloaders["train"])):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # 파라미터 경사도 초기화
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            # 역전파
            loss.backward()
            # 최적화
            optimizer.step()
            
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
        
        avg_loss = loss_train / dataset_sizes["train"]
        avg_acc = acc_train.double() / dataset_sizes["train"]
        
        # validation
        model.train(False)
        model.eval()
            
        for i, data in tqdm(enumerate(dataloaders["val"])):
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.item()
            acc_val += torch.sum(preds == labels.data)
        
        avg_loss_val = loss_val / dataset_sizes["val"]
        avg_acc_val = acc_val.double() / dataset_sizes["val"]
        
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
ef_model = train_model(ef_model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)
# 모델 저장
torch.save(ef_model.state_dict(), 'model.pt')

## 모델 평가

In [ ]:
eval_model(ef_model, criterion)

In [ ]:
visualize_model(ef_model, num_images=8)

## 제출

In [ ]:
# 모델 준비
#model = VGG11(len(class_names))
#model.load_state_dict(torch.load("model.pt"))

ef_model.to(device)
ef_model.eval()

In [ ]:
# #modelA load
# modelA = geffnet.efficientnet_b3(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
# modelA.train()

# # 이전 레이어 학습 정지
# for param in modelA.parameters():
#     param.require_grad = True #원래 False

    
# # 새로운 레이어 추가
# num_features = modelA.classifier.in_features
# features = list(modelA.classifier.children())[:-1]
# features.extend([nn.Linear(num_features, len(class_names))])
# modelA.classifier = nn.Sequential(*features) # classifier 변경

# modelA.load_state_dict(torch.load("../input/ensemble-models/model_best.pt"))
# modelA.to(device)


# #modelB load
# modelB = geffnet.efficientnet_b3(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
# modelB.train()

# # 이전 레이어 학습 정지
# for param in modelB.parameters():
#     param.require_grad = True #원래 False

    
# # 새로운 레이어 추가
# num_features = modelB.classifier.in_features
# features = list(modelB.classifier.children())[:-1]
# features.extend([nn.Linear(num_features, len(class_names))])
# modelB.classifier = nn.Sequential(*features) # classifier 변경

# modelB.load_state_dict(torch.load("../input/ensemble-models/model_2.pt"))
# modelB.to(device)



# #modelC load
# modelC = geffnet.efficientnet_b3(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
# modelC.train()

# # 이전 레이어 학습 정지
# for param in modelC.parameters():
#     param.require_grad = True #원래 False

    
# # 새로운 레이어 추가
# num_features = modelC.classifier.in_features
# features = list(modelC.classifier.children())[:-1]
# features.extend([nn.Linear(num_features, len(class_names))])
# modelC.classifier = nn.Sequential(*features) # classifier 변경

# modelC.load_state_dict(torch.load("../input/ensemble-models/model_3.pt"))
# modelC.to(device)


In [ ]:
# test 이미지 경로
paths = []
target_dir = "../input/track02-dataset/hallym-2020-2-aicomp02/test"
#target_dir = "../input/new-data/test"
for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
    for fname in sorted(fnames):
        path = os.path.join(target_dir, fname)
        paths += [path]

In [ ]:
predictions = []

batch_size = 16
for i in tqdm(range(round((len(paths) / batch_size) + 0.5))):
    # 이미지 batch 처리
    batch = paths[i*batch_size:(i+1)*batch_size]
    inputs = []
    for j, path in enumerate(batch):
        img = Image.open(path).convert('RGB')
        img = transform(img)
        inputs.append(img)

    inputs = torch.stack(inputs).to(device)

    # prediction
    preds = ef_model(inputs)
#     pred1 = modelA(inputs)
#     pred2 = modelB(inputs)
#     pred3 = modelC(inputs)
#     preds = (pred1 + pred2 + pred3) / 3
    predictions += [preds.detach().cpu().numpy()]
    del preds
    torch.cuda.empty_cache()

In [ ]:
# write csv
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["Id","Category"])
    for i, batch in enumerate(predictions):
        for j, pred in enumerate(batch):
            row = [os.path.basename(paths[i * batch_size + j]), class_names[pred.argmax()]]
            writer.writerow(row)